# 1 - Importa bibliotecas

In [1]:
import os
import gym
import numpy as np
import websocket
import json
import time
import random
import psutil

# 2 - Cria funções de encoding/decoding em json

In [2]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)                
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [3]:
def encode_json(identifier, reward, done, states):
    data = {}
    data['identifier'] = identifier
    data['reward'] = reward
    data['done'] = done
    data['states'] = states
    json_data = json.dumps(data, cls=NumpyEncoder)       
    return json_data

# 3 - Define função do ambiente de treino

In [4]:
def enviroment(data, gripClose, reward, done, canTimer, canBox, intervalbox, ConveyorFinish, SensorBox, BoxBackward,
                   ToClose, ToCloseTolet, picked, step, GantryYposition, GantryZposition, canPosition, erroPosicaoBarreira):
        
        GantryYposition = GantryYposition
        GantryZposition = GantryZposition
        rowFinished = False
        message = ''
        erroPosicaoBarreira = False
        GantryYpositionBackup = GantryYposition
        GantryZpositionBackup = GantryZposition
        
        #print(data)
        if data['actions'][0] == 0:
            gripClose = False            
            
        if data['actions'][0] == 1:
            gripClose = True
            
        if data['actions'][0] == 2:        
            GantryZposition = 0
            GantryYposition = 0
            
        if data['actions'][0] == 3:        
            GantryZposition = 200
            GantryYposition = 0              
            
        if data['actions'][0] == 4:
            GantryZposition = 0
            GantryYposition = 430
                    
        if data['actions'][0] == 5:
            GantryZposition = 200
            GantryYposition = 430
                    
        if data['actions'][0] == 6:
            GantryZposition = 0
            GantryYposition = 530
                    
        if data['actions'][0] == 7:
            GantryZposition = 200
            GantryYposition = 530
            
        if data['actions'][0] == 8:
            GantryZposition = 0
            GantryYposition = 630 
            
        if data['actions'][0] == 9:
            GantryZposition = 200
            GantryYposition = 630 
            
        if data['actions'][0] == 10:
            rowFinished = True
        reward = 0    
            
        if (GantryZposition > 180  and 
            ((GantryYpositionBackup == 0 and GantryYposition > 420) or (GantryYposition == 0 and GantryYpositionBackup > 420))):
            erroPosicaoBarreira = True
        if (GantryZposition < 180  and 
            ((GantryYpositionBackup == 0 and GantryYposition > 420) or (GantryYposition == 0 and GantryYpositionBackup > 420))):
            erroPosicaoBarreira = False
        
        gripYStart = True
        gripZStart = True
            
        
        step += 1
        canTimer += 1
        def reward_canInBox(canPosition, step, reward, canBox):
            if GantryZposition < 180:
                reward = (400 * canPosition.count(1)) -  step/2
                print('!!!-->>Can despejada || canPosition: ',  canPosition, '  || interval:  ', step  ,'  || reward:  ' ,reward)
            else:
                reward = (4000 * canPosition.count(1)) - step
                print('!!!>>>-->>Sucesso! || canPosition: ',  canPosition, '  || interval:  ', step  ,'  || reward:  ' ,reward)               
            #reward = random.choice([1000,1000,1000,1000,1000,1000]) 
            picked = False
            step = 0
            canBox += 1
            ToCloseTolet = False
            return reward, picked, step, canBox
        
        if gripClose:
            GripperClose = True
        else:
            GripperClose = False
      
        GantryYatDestination = True
        GantryZatDestination = True
        GantryYdriving = gripYStart
        GantryZdriving = gripZStart
        #reward = -(10 + 2 * step)
        
        
        
        if erroPosicaoBarreira:          
            #reward = -10
            #done = True
            #print("/////// ---- erro de percurso ----- \\\\\\\-")
            pass
            
            
        
        if not erroPosicaoBarreira:
            if canTimer > 9:
                ConveyorFinish = True        
     
            if GantryYposition < 20 and (GantryZposition  > 190 and GantryZposition  <220) and GantryZpositionBackup < 180:            
                if ConveyorFinish and gripClose == False and ToClose == False:
                    ToClose = True
                    #reward = 5
                    #reward = random.choice([0,0,0,0,100,100]) 
                    #print("!! - Conveyor finished -- !!")
                if not ConveyorFinish:                    
                    #pass
                    reward = -500
                    #done = True
                    #print("**** - Conveyor not finished -- ****")
            
            
            #print('cantimer: '  canTimer '  || interval:  ' step  '  || reward:  ' reward)
            #step = 0
            
            if GantryYposition > 20 or GantryZposition < 190:
                ToClose = False

            if GantryYposition < 20 and (GantryZposition  > 190 and GantryZposition  <220) and ToClose and gripClose and ConveyorFinish:
                canTimer = random.randint(0, 9)
                ConveyorFinish = False
                picked = random.choice([True, True, True, True])        
                #step = 0

            #if picked and (GantryZposition  > 190 and GantryZposition  <220):
                #reward = random.choice([0,0,(40 + (GantryYposition/20) + ((GantryZposition -180)/6))])
                #print('reward picked || position: -'  GantryYposition '  'GantryZposition'-  || interval:  ' step  '  || reward:  ' reward)

            if canPosition != [1,1,1] and rowFinished:
                #canPosition = [0,0,0]
                #print('__---+++ ----------finish errado  --------------->>  can box: ', canBox)
                reward = -100 -  ( step)
            
            if canPosition == [1,1,1] and rowFinished:
                canPosition = [0,0,0]
                print('!!!---!!! ----------linha completa  --------------->>  can box: ', canBox)
                reward = +2000 -  (10 * step)
                

                
            if picked and GantryYposition >= 430  and gripClose == False and not BoxBackward and ToCloseTolet and canBox < 10:
                if (GantryYposition == 430 and canPosition[0] == 1) or (GantryYposition == 530 and canPosition[1] == 1) or (GantryYposition == 630 and canPosition[2] == 1):
                    reward = -100 - (5 * step)
                    #done = True
                    picked = False
                    print('%penalidade can mesma posicao || canPosition: ',  canPosition, '  || interval:  ', step  ,'  || reward:  ' ,reward)
                    #step = 0
                if GantryYposition == 430 and canPosition[0] == 0:
                    canPosition[0] = 1
                    reward, picked, step, canBox = reward_canInBox(canPosition, step, reward, canBox)                
                if GantryYposition == 530 and canPosition[1] == 0:
                    canPosition[1] = 1
                    reward, picked, step, canBox = reward_canInBox(canPosition, step, reward, canBox)                
                if GantryYposition == 630 and canPosition[2] == 0:
                    canPosition[2] = 1
                    reward, picked, step, canBox = reward_canInBox(canPosition, step, reward, canBox)
                   

            if picked and GantryYposition >= 430  and ((GantryZposition > 180 and GantryZpositionBackup < 180)):
                ToCloseTolet = True
                #reward = 20
            
            if GantryYposition < 430:
                ToCloseTolet = False
                


                

            if picked and (GantryYposition < 430) and gripClose == False:
                #reward = -(20 - (GantryYposition/50) )      
                picked = False
                #print('%penalidade posicao || cantimer: ',  canTimer,  '  || interval:  ', step,  '  || reward:  ', reward)
                #step = 0

            if picked and BoxBackward and gripClose == False:
                reward = -400        
                picked = False
                print('%penalidade BoxConveyor || cantimer: ',  canTimer,  '  || interval:  ', step , '  || reward:  ', reward)
                #step = 0
                #done = True
                

            if canBox == 9 and canPosition == [0,0,0]:
                intervalbox += 1
                BoxBackward = True
                SensorBox = False
                print('intervalBox:  ', intervalbox ,'  |  interval:  ', step)

            if intervalbox == 30: 
                done = True
                SensorBox = True
                reward = +6000 - (60 * step)
                print('*****COMPLETOU A TAREFA!!****** || can position: ',  canPosition, '  || interval:  ', step  ,'  || reward:  ' ,reward)
                canTimer = random.randint(0, 9) 
                canBox = 0
                intervalbox = random.randint(15, 29)
                ConveyorFinish = False
                ToClose = False
                picked = False
                step = 0
                BoxBackward = False
                canPosition = [0,0,0]
                erroPosicaoBarreira = False
            
            
            
        states = [ConveyorFinish, picked, round((GantryYposition/630),2), round((GantryZposition/210),2), gripClose, BoxBackward]
                  
                      
        if step > 500:            
            done = True
        
        message = encode_json('Handling', reward, done, states)    
        return message, reward, done, canTimer, canBox, intervalbox, ConveyorFinish, SensorBox, BoxBackward, ToClose, ToCloseTolet, picked, step, GantryYposition, GantryZposition, gripClose, canPosition, erroPosicaoBarreira

# 4 - Verifica se o servidor socket está ativo

In [9]:
server_found = False

for conn in psutil.net_connections():
    if conn.laddr.port == 12000 and conn.status == 'LISTEN':
        print("Um servidor socket está sendo executado na porta 12000.")
        server_found = True

if not server_found:
    print("Nenhum servidor socket está sendo executado na porta 12000.")

Um servidor socket está sendo executado na porta 12000.


# 5 - Ativa o ambiente para treino

In [27]:
ws = websocket.WebSocket()
ws.connect("ws://127.0.0.1:12000")
time.sleep(1)
reward = 0
done = False
canTimer = 8 #random.randint(0, 9) 
canBox = 0
intervalbox = random.randint(15, 29)
ConveyorFinish = False
ToClose = False
ToCloseTolet = False
picked = False
done = False
step = 0
message =''
canPosition = [0,0,0]
erroPosicaoBarreira = False
    
GantryYposition = 0
    
GantryZposition = 0
    
gripClose = False
SensorBox = False
BoxBackward = False
while True:
    data =''
    data = json.loads(ws.recv())
    if data['identifier']  == 'Handling_manual':
        message, reward, done, canTimer, canBox, intervalbox, ConveyorFinish, SensorBox, BoxBackward, ToClose, ToCloseTolet, picked, step, GantryYposition, GantryZposition, gripClose, canPosition, erroPosicaoBarreira = enviroment(data,
        gripClose, reward, done, canTimer, canBox, intervalbox, ConveyorFinish, SensorBox, BoxBackward, ToClose, ToCloseTolet, picked, step, GantryYposition, GantryZposition, canPosition, erroPosicaoBarreira)
                                      
        #print (message)
        ws.send(message)
        if done:
            done = False
            reward = 0
            canTimer = random.randint(0, 9) 
            canBox = 0
            intervalbox = random.randint(15, 29)
            ConveyorFinish = False
            ToClose = False
            picked = False
            done = False
            step = 0
            BoxBackward = False
            message =''
            canPosition = [0,0,0]
            GantryYposition = 0
            GantryZposition = 0
            erroPosicaoBarreira = False
            
            

ConnectionResetError: [WinError 10054] Foi forçado o cancelamento de uma conexão existente pelo host remoto